<p style="float:right">
<img src="images/logos/cu.png" style="display:inline" />
<img src="images/logos/cires.png" style="display:inline" />
<img src="images/logos/nasa.png" style="display:inline" />
<img src="images/logos/nsidc_daac.png" style="display:inline" />
</p>

# Python, Jupyter & pandas: Module 3

## Viewing geolocated data

In Module 2, we retrieved NSIDC snow cover data in the northern hemisphere from an OPeNDAP server. Let's pull that data again for use in this notebook:

In [ ]:
import netCDF4
import numpy as np

url_min = ('http://n5eil01u.ecs.nsidc.org/opendap/hyrax/MEASURES/NSIDC-0530.001/2012.01.01/'
           'nhtsd25e2_20120101_v01r01.nc?'
           'latitude[0:1:719][0:1:719],'
           'longitude[0:1:719][0:1:719],'
           'merged_snow_cover_extent[0:1:0][0:1:719][0:1:719]'
)

dataset_min = netCDF4.Dataset(url_min)

latitude_min = dataset_min.variables['latitude'][:, :]
longitude_min = dataset_min.variables['longitude'][:, :]
msce_min = dataset_min.variables['merged_snow_cover_extent'][0, :, :]

And now let's pull in some graphics support now and get ready to plot our snow cover data.

In [ ]:
%matplotlib inline
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

It will be helpful to have an underlying map image to better interpret our data, so let's be sure we can at least do that with `matplotlib`'s [`Basemap`](http://matplotlib.org/basemap/api/basemap_api.html?highlight=basemap#module-mpl_toolkits.basemap):

In [ ]:
plt.figure(figsize=(10, 10))
m_blank = Basemap(projection='npstere', boundinglat=45, lon_0=300)
m_blank.drawcoastlines()

We can ask for more interesting background imagery (we won't, in general, in this tutorial, in order to focus on the data instead of the map):

In [ ]:
plt.figure(figsize=(10, 10))
m_imagery = Basemap(projection='npstere', boundinglat=45, lon_0=300)
m_imagery.drawcoastlines()
m_imagery.etopo()

Let's take a quick look back at the metadata for our Merged Snow Cover Extent variable:

In [ ]:
dataset_min.variables['merged_snow_cover_extent']

So, the snow cover variable is categorical (discrete), and consists of integers whose values represent snow cover from different sources, snow-free land, permanent ice-covered land, ocean, or a fill value.

As a reality check, let's find the would-be ocean points and plot them on our map.

In the following cell, we pass longitude and latitude values into our `Basemap` object `m`, which [translates them to `x` and `y` Cartesian coordinates appropriate to our map](http://matplotlib.org/basemap/users/mapcoords.html). Translating from a sphere (more or less) onto a flat map inevitably introduces some distortion, and each _projection_ (here we use a North Polar Stereographic projection) has its pros and cons.

Note that `x`, `y`, `longitude_min` and `latitude_min` are all arrays. Basemap can operate on arrays as readily as it can on single values.

In [ ]:
plt.figure(figsize=(10, 10))
m0 = Basemap(projection='npstere', boundinglat=45, lon_0=300)
m0.drawcoastlines()

row0, col0 = np.where(msce_min == 40)
x0, y0 = m0(longitude_min[row0, col0], latitude_min[row0, col0])
m0.scatter(x0, y0, s=1, color='SeaGreen')

Seems legit.

Now, where's the snow? Values 10 through 16 in the merged snow cover variable all indicate snow, so we'll accept anything in that range:

In [ ]:
plt.figure(figsize=(10, 10))
m1 = Basemap(projection='npstere', boundinglat=45, lon_0=300)
m1.drawcoastlines()

row1, col1 = np.where((msce_min >= 10) & (msce_min <= 16))
x1, y1 = m1(longitude_min[row1, col1], latitude_min[row1, col1])
m1.scatter(x1, y1, s=1, color='Cyan')

How about snow **or** permanent ice covered land?

In [ ]:
plt.figure(figsize=(10, 10))
m2 = Basemap(projection='npstere', boundinglat=45, lon_0=300)
m2.drawcoastlines()

snow2 = (msce_min >= 10) & (msce_min <= 16)
ice2 = (msce_min == 30)

row_snow2, col_snow2 = np.where(snow2)
row_ice2, col_ice2 = np.where(ice2)

x_snow2, y_snow2 = m2(longitude_min[row_snow2, col_snow2], latitude_min[row_snow2, col_snow2])
x_ice2, y_ice2 = m2(longitude_min[row_ice2, col_ice2], latitude_min[row_ice2, col_ice2])

m2.scatter(x_ice2, y_ice2, s=1, color='Blue')
m2.scatter(x_snow2, y_snow2, s=1, color='Cyan')

At the end of Module 2, we tried to use OPeNDAP to retrieve data only in the area of Iceland. Let's execute that query again:

In [ ]:
iceland_url = ('http://n5eil01u.ecs.nsidc.org/opendap/hyrax/MEASURES/NSIDC-0530.001/2012.01.01/'
               'nhtsd25e2_20120101_v01r01.nc?'
               'latitude[453:1:476][310:1:338],'
               'longitude[453:1:476][310:1:338],'
               'merged_snow_cover_extent[0:1:0][453:1:476][310:1:338]'
)

iceland_dataset = netCDF4.Dataset(iceland_url)

iceland_latitude = iceland_dataset.variables['latitude'][:, :]
iceland_longitude = iceland_dataset.variables['longitude'][:, :]
iceland_msce = iceland_dataset.variables['merged_snow_cover_extent'][0, :, :]

And now let's plot it on a map and see if the result makes sense:

In [ ]:
plt.figure(figsize=(10, 10))
m3 = Basemap(projection='npstere', boundinglat=45, lon_0=300)
m3.drawcoastlines()

snow3 = (iceland_msce >= 10) & (iceland_msce <= 16)
ice3 = (iceland_msce == 30)

row_snow3, col_snow3 = np.where(snow3)
row_ice3, col_ice3 = np.where(ice3)

x_snow3, y_snow3 = m3(iceland_longitude[row_snow3, col_snow3], iceland_latitude[row_snow3, col_snow3])
x_ice3, y_ice3 = m3(iceland_longitude[row_ice3, col_ice3], iceland_latitude[row_ice3, col_ice3])

m3.scatter(x_snow3, y_snow3, s=1, color='Cyan')
m3.scatter(x_ice3, y_ice3, s=1, color='Blue')

Looks about right, but let's get a closer look. Can we plot a map of just Iceland?

In [ ]:
plt.figure(figsize=(10, 10))
m4 = Basemap(projection='merc', llcrnrlon=-25, llcrnrlat=63, urcrnrlon=-13, urcrnrlat=67, resolution='i')
m4.drawcoastlines()

Let's plot the snow- and ice-cover points. We can re-use the snow/ice row/column variables we computed previously, but need to recompute the coordinates since we are using a different map projection (Mercator, instead of North Polar Stereographic).

In [ ]:
plt.figure(figsize=(10, 10))
m5 = Basemap(projection='merc', llcrnrlon=-25, llcrnrlat=63, urcrnrlon=-13, urcrnrlat=67, resolution='i')
m5.drawcoastlines()

x_snow5, y_snow5 = m5(iceland_longitude[row_snow3, col_snow3], iceland_latitude[row_snow3, col_snow3])
x_ice5, y_ice5 = m5(iceland_longitude[row_ice3, col_ice3], iceland_latitude[row_ice3, col_ice3])

m5.scatter(x_ice5, y_ice5, s=100, color='Blue')
m5.scatter(x_snow5, y_snow5, s=100, color='Cyan')

If we add some (admittedly lo-res) background imagery, we can see that the permanent ice is located in what appear to be sensible places:

In [ ]:
plt.figure(figsize=(10, 10))
m6 = Basemap(projection='merc', llcrnrlon=-25, llcrnrlat=63, urcrnrlon=-13, urcrnrlat=67, resolution='i')
m6.scatter(x_snow5, y_snow5, s=100, color='Cyan')
m6.scatter(x_ice5, y_ice5, s=100, color='Blue')
m6.bluemarble()

There are certainly more sophisticated ways to visualize these data. In the `module-3-extra` notebook in the `extra` folder, there's an example of something a bit nicer. But hopefully this gives an idea of what's possible.

In Module 4, we'll look at using pandas to do some data analysis.